# LLM API Testing

## Validate game moves

In [6]:
# Define a function to validate a move
def is_valid_move(shape, move_from, move_to):
    rows = len(shape)
    cols = len(shape[0])
    
    # Check if move_from and move_to are within bounds
    if not (0 <= move_from[0] < rows and 0 <= move_from[1] < cols):
        return False
    if not (0 <= move_to[0] < rows and 0 <= move_to[1] < cols):
        return False
    
    # Check if there is a square at move_from and no square at move_to
    if shape[move_from[0]][move_from[1]] == 0 or shape[move_to[0]][move_to[1]] == 1:
        return False
    
    # Check if move_from is not in the center
    center_row = rows // 2
    center_col = cols // 2
    if move_from == (center_row, center_col):
        return False
    
    # Check if move_to is adjacent to an existing square (not diagonally)
    adjacent_positions = [
        (move_to[0] - 1, move_to[1]),  # Up
        (move_to[0] + 1, move_to[1]),  # Down
        (move_to[0], move_to[1] - 1),  # Left
        (move_to[0], move_to[1] + 1)   # Right
    ]
    
    for pos in adjacent_positions:
        if 0 <= pos[0] < rows and 0 <= pos[1] < cols and shape[pos[0]][pos[1]] == 1:
            return True
    
    return False

# Example usage:
shape = [
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]
]
move_from = (0, 4)
move_to = (2, 2)
print(is_valid_move(shape, move_from, move_to))  # Output: True (adjacent)

move_from = (0, 4)
move_to = (1, 4)
print(is_valid_move(shape, move_from, move_to))  # Output: False (not adjacent)



True
False


In [5]:
tt

True


# Get response from LLM

In [3]:
import json
import datetime



# Set your OpenAI API key
# import openai
# openai.api_key = 'XXXXXX'

# Define a function to get a response from the OpenAI API
def get_openai_response_fake(user_prompt, prompt):
    # use user_prompt as file_name to get the full game instructions
    try:
        prompt = {k:v for k,v in prompt.items() if v}  # Remove empty values from prompt (e.g., initial prompt)
        input = {
            'user_prompt': user_prompt,    # Full game instructions
            'prompt': json.dumps(prompt),  # Current move instructions (dict dumped to string)
        }

        # Todo: actually call API
        response = "2 1022"  # Hardcoded fake response
        output = input.copy()
        output['current_shape'] = prompt.get('current_shape')
        output['last_shape'] = prompt.get('last_shape')
        output['response'] = response
        output['timestamp'] = datetime.datetime.now().isoformat()
        output['valid_move'] = True # TODO Hardcoded, but need to write a function for checking
        return output
    except Exception as e:
        return f"An error occurred: {e}"
    
# Valid moves example
# 1023 -> 2 1022 -> 4 1020 512

# Single example
user_prompt = "instructions/version1.txt"
prompt={
    "current_shape": "1023", 
    "last_shape": ""
}
get_openai_response_fake(user_prompt, prompt)

{'user_prompt': 'instructions/version1.txt',
 'prompt': '{"current_shape": "1023"}',
 'current_shape': '1023',
 'last_shape': None,
 'response': '2 1022',
 'timestamp': '2025-01-30T14:03:08.350019',
 'valid_move': True}

In [4]:
# Interaction loop for game

THRESHOLD_TOTAL_MOVES = 10
THRESHOLD_TOTAL_RETRIES = 10

user_prompt = "instructions/version1.txt"
prompt={
    "current_shape": "1023", 
    "last_shape": ""
}

move_count = 0
retry_count = 0

while move_count < THRESHOLD_TOTAL_MOVES and retry_count < THRESHOLD_TOTAL_RETRIES:
    output = get_openai_response_fake(user_prompt, prompt)
    print(output)
    if output['valid_move']:
        move_count += 1
        retry_count = 0
        prompt['current_shape'] = output['response']
    else:
        retry_count += 1
        print(f"Invalid move, retrying... {retry_count} / {THRESHOLD_TOTAL_RETRIES}")
    
    # TODO: save response dictionary to file (append as you go)


{'user_prompt': 'instructions/version1.txt', 'prompt': '{"current_shape": "1023"}', 'current_shape': '1023', 'last_shape': None, 'response': '2 1022', 'timestamp': '2025-01-30T14:03:08.384481', 'valid_move': True}
{'user_prompt': 'instructions/version1.txt', 'prompt': '{"current_shape": "2 1022"}', 'current_shape': '2 1022', 'last_shape': None, 'response': '2 1022', 'timestamp': '2025-01-30T14:03:08.384932', 'valid_move': True}
{'user_prompt': 'instructions/version1.txt', 'prompt': '{"current_shape": "2 1022"}', 'current_shape': '2 1022', 'last_shape': None, 'response': '2 1022', 'timestamp': '2025-01-30T14:03:08.385012', 'valid_move': True}
{'user_prompt': 'instructions/version1.txt', 'prompt': '{"current_shape": "2 1022"}', 'current_shape': '2 1022', 'last_shape': None, 'response': '2 1022', 'timestamp': '2025-01-30T14:03:08.385114', 'valid_move': True}
{'user_prompt': 'instructions/version1.txt', 'prompt': '{"current_shape": "2 1022"}', 'current_shape': '2 1022', 'last_shape': None,